In [2]:

from shapely.geometry import Point
import pandas as pd
import math
import geopandas as gpd
import re
import numpy as np
from rapidfuzz import process, fuzz
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score

df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')
pd.set_option('display.max_columns', None)


C:\Users\gabri\AppData\Local\Temp\ipykernel_10232\3036778390.py:12: DtypeWarning: Columns (11,12,17,18,20,21,22,23,25,26,27,28,29,30,31,32,33,36,37,39,40,42,43,44,45,46,47,48,49,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('dados/SpSafe_2022.csv', delimiter=';')


<div style="color:Blue; text-align:center;">
<h1>Pergunta: É possível prever a quantidade de crimes anual baseado na quantidade populacional, crimes por 100mil habitantes, IDMH e o índice de Ideb, usando regressão linear?</h1>
</div>

In [4]:
df_cidades = pd.read_csv('dados/SpSafe_2022_cidades.csv', delimiter=';')
idmh = gpd.read_file('dados/teste/IDHM.shp')
df_cidades['IDMH'] = 0
df_cidades['cod_ibge'] = 0
df_cidades['cod_ra'] = 0
df_cidades['reg_administrativa'] = 0
df_cidades['reg_metropolitanas'] = 0


indices = df_cidades[df_cidades["QUANTIDADE_POPULACAO"] == 0].index
df_cidades = df_cidades.drop(indices)



<div style = "text-align:center";>
    <p>Preparando os dados </p>
</div>

In [15]:
df_cidades['CIDADE'] = df_cidades['CIDADE'].replace({
    'Santo Antônio da Alegria': 'Santo Antonio da Alegria',
    'Itaoca': 'Itaóca',
    'São Luiz do Paraitinga': 'São Luís do Paraitinga',
    'Santo Antônio de Posse':'Santo Antonio de Posse',
    'Santo Antônio do Jardim' : 'Santo Antonio do Jardim',
    'Mogi Mirim': 'Moji Mirim',
    'Tarabai': 'Tarabaí',
    'Santa Lúcia': 'Santa Lucia',
    'Santa Rosa de Viterbo' : 'Santa Rosa do Viterbo',
    'Santo Antônio do Aracanguá' : 'Santo Antonio do Aracanguá',
    'Uchoa': 'Uchôa'

})


<div style = "text-align:center";>
    <p>Dados adquiridos também via fundação SEADE. <p>
</div>

In [5]:
exm = pd.read_csv("dados/teste/codigos_municipios_regioes.csv", encoding="latin1", delimiter=";")
exm.head()


,cod_ibge,municipios,cod_ra,reg_administrativa,reg_metropolitanas,cod_ibge6,pop2020,porte_pop2020,cod_drs,drs
0,3500000,Estado sem especificação,Estado sem especificação,Estado sem especificação,Estado sem especificação,350000,Estado sem especificação,Estado sem especificação,Estado sem especificação,Estado sem especificação
1,3500105,Adamantina,691,RA de Presidente Prudente,Demais municípios,350010,33894,Mais de 20 mil até 50 mil,5,Marília
2,3500204,Adolfo,689,RA de São José do Rio Preto,RM São José do Rio Preto,350020,3447,Até 5 mil,15,São José do Rio Preto
3,3500303,Aguaí,686,RA de Campinas,Demais municípios,350030,35608,Mais de 20 mil até 50 mil,14,São João da Boa Vista
4,3500402,Águas da Prata,686,RA de Campinas,Demais municípios,350040,7797,Mais de 5 mil até 10 mil,14,São João da Boa Vista


In [6]:
df_cidades.head()

,CIDADE,QUANTIDADE_CRIMES,QUANTIDADE_POPULACAO,CRIMES_POR_100K,IDMH,cod_ibge,cod_ra,reg_administrativa,reg_metropolitanas
0,São Paulo,251285,11429865,2198.0,0,0,0,0,0
1,Santo André,16033,752784,2130.0,0,0,0,0,0
2,Campinas,13731,1140133,1204.0,0,0,0,0,0
3,Guarulhos,12815,1293335,991.0,0,0,0,0,0
4,São Bernardo do Campo,11086,811198,1367.0,0,0,0,0,0


In [16]:
for indice, linha in df_cidades.iterrows():
    for indice_e, linha_e in exm.iterrows():
        if(linha['CIDADE'] == linha_e['municipios']):
            df_cidades.at[indice, 'cod_ibge'] = linha_e['cod_ibge']
            df_cidades.at[indice, 'cod_ra'] = linha_e['cod_ra']
            df_cidades.at[indice, 'reg_administrativa'] = linha_e['reg_administrativa']
            df_cidades.at[indice, 'reg_metropolitanas'] = linha_e['reg_metropolitanas']
          

In [13]:
for indice, linha in df_cidades.iterrows():
    for indice_idmh, linha_idmh in idmh.iterrows():
        if linha['cod_ibge'] == linha_idmh['Cod_ibge']:
            df_cidades.at[indice, 'IDMH'] = linha_idmh['IDMH']
            break

C:\Users\gabri\AppData\Local\Temp\ipykernel_10232\1660612305.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.8' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_cidades.at[indice, 'IDMH'] = linha_idmh['IDMH']


In [31]:
ideb = pd.read_csv('educacaoideb.csv', delimiter=';', encoding='latin1')
df_cidades['ideb2019'] = 0
ideb['Cod'] = ideb['Cod'].dropna().astype(int)

In [ ]:
for indice, linha in df_cidades.iterrows():
    for indice_ideb, linha_ideb in ideb.iterrows():
        if linha['cod_ibge'] == linha_ideb['Cod']:
            if linha_ideb['Rede'] == 'PÃºblica':
                df_cidades.at[indice, 'ideb2019'] = linha_ideb['IDEB 2019']
                break


In [39]:
for indice, linha in df_cidades.iterrows():
    if pd.isna(linha['ideb2019']):
        df_cidades.at[indice, 'ideb2019'] = 0

In [55]:
df_cidades_encoded = pd.get_dummies(df_cidades, columns=["CIDADE"])


In [40]:
df_cidades.to_csv('SpSafe_2022_cidades.csv', index=False)

<div style = "text-align:center";>
    <p>Criando a o modelo baseado nos dados adquiridos <p>
</div>

In [57]:

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Estilo dos gráficos
sns.set(style="whitegrid")

# Remover linhas com valores infinitos ou NaN
df_cidades_clean = df_cidades.replace([np.inf, -np.inf], np.nan).dropna(subset=['QUANTIDADE_POPULACAO', 'CRIMES_POR_100K', 'QUANTIDADE_CRIMES'])

# Separar variáveis
X = df_cidades_clean[['QUANTIDADE_POPULACAO', 'CRIMES_POR_100K', 'IDMH', 'ideb2019']]
y = df_cidades_clean['QUANTIDADE_CRIMES']

# Dividir em treino (80%) e teste (20%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinar o modelo com os dados de treino
model = LinearRegression()
model.fit(X_train, y_train)

# Fazer previsões no conjunto de teste
y_pred = model.predict(X_test)

# Avaliação
resultados = {
    "Coeficiente (a)": [model.coef_[0]],
    "Intercepto (b)": [model.intercept_],
    "R² (teste)": [r2_score(y_test, y_pred)],
    "Erro médio quadrático (MSE)": [mean_squared_error(y_test, y_pred)]
}

tabela_resultados = pd.DataFrame(resultados)
display(tabela_resultados.round(2))


,Coeficiente (a),Intercepto (b),R² (teste),Erro médio quadrático (MSE)
0,0.02,4656.43,0.32,1384953.28
